# Project #1
## Data Visualization (LECD) 2024/25

#### Student name: Stéfano Cardoso Bernardes Nascimento

## Instructions
- Do not modify the structure of this notebook!
- Fill-in the corresponding cells with the code or text depending on the objective
- You can use this template as a working notebook, or copy&paste your final code here
- It is recommended that you export and submit the rendered version of the notebook
- Submit the notebook not later than the date specified on the Inforestudante webpage of the course


## Data
#### Provide a brief description of the chosen dataset and the attributes. If any cleaning or filtering was done, also describe it in this section

O arquivo do dataset escolhido se chama 'netflix_titles.csv', é sobre os detalhes dos títulos adicionados na plataforma da Netflix ao longo dos anos. Os atributos escolhidos para serem analisados foram o 'type', 'date_added', 'release_year' e 'duration'. O atributo 'type' refere-se ao tipo de produção audiovisual, pode a ser filmes ou Tv Shows. O atributo 'date_added' refere-se a data que uma determinada mídia foi adicionada na Netflix, variando ao longo dos anos de 2008 a 2021. O atributo 'duration' refere-se ao tempo de duração dessas mídias, pode a ser em minutos se forem filmes ou em quantidade de temporadas se forem TV Shows. Por fim, o atributo 'release_year' define a idade média de um conjunto de títulos lançado num determinado ano. Foi realizado um processo de limpeza dos dados para eliminar campos com espaços 'extras', também foi realizado uma filtragem dos dados especificamente nas colunas 'date_added' e 'duration', para extrair em cada célula apenas valores numéricos. Isso permitiu trabalhar a contagem e os cálculos desses dados de forma mais segura e eficiente.

In [9]:
# import your dataset in this section (cell)
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Realiza a importação do dataset.
df=pd.read_csv('netflix_titles.csv')

# Limpa os espaços extras de todas as colunas.
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.str.strip())

# Converte a coluna 'release_year' para numérico, se ainda não estiver.
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

# Cria a coluna 'year_added' para receber apenas os anos dos campos da coluna date_added.
df['date_added'] = pd.to_datetime(
    df['date_added'], 
    format='%B %d, %Y', 
    errors='coerce'
)
df['year_added'] = df['date_added'].dt.year

# Remove os registros nulos da coluna 'year_added'.
df = df.dropna(subset=['year_added'])

# Extrai apenas os números da coluna 'duration'.
df['duration_num'] = df['duration'].str.extract(r'(\d+)').astype(float)

## Visualization model #1

In [10]:
# Define a lista de anos para o eixo X, garantindo que esteja ordenada.
anos = sorted(df['year_added'].unique())

# Criação de variáveis que recebem os campos do atributo type.
movies_df = df[df['type'] == 'Movie']
tv_shows_df = df[df['type'] == 'TV Show']

# Realiza a contagem da quantidade de filmes e a média de duração e release_year de acordo com o ano.
movies_grouped = movies_df.groupby('year_added').agg(
    counts=('title', 'count'),
    avg_duration=('duration_num', 'mean'),
    avg_release_year=('release_year', 'mean')  # Média de release_year
).reset_index()

# Realiza a contagem da quantidade de Tv Shows e a média de temporadas e release_year de acordo com o ano.
tv_shows_grouped = tv_shows_df.groupby('year_added').agg(
    counts=('title', 'count'),
    avg_seasons=('duration_num', 'mean'),
    avg_release_year=('release_year', 'mean') 
).reset_index()

# Normaliza os valores médios para o intervalo entre 0 e 1.
movies_grouped['normalized_avg_duration'] = (
    movies_grouped['avg_duration'] - movies_grouped['avg_duration'].min()
) / (movies_grouped['avg_duration'].max() - movies_grouped['avg_duration'].min())

tv_shows_grouped['normalized_avg_seasons'] = (
    tv_shows_grouped['avg_seasons'] - tv_shows_grouped['avg_seasons'].min()
) / (tv_shows_grouped['avg_seasons'].max() - tv_shows_grouped['avg_seasons'].min())

# Define uma largura fixa para as barras dos gráficos.
fixed_bar_width = 0.4

# Cria a figura (combinação do Layout e a definição dos dados).
fig = go.Figure()

# Adiciona na legenda a barra de cor referência (filme) com escala (valor média em minutos) e hover com média de release_year.
fig.add_trace(
    go.Bar(
        x=movies_grouped['year_added'],
        y=movies_grouped['counts'],
        name='Filmes',
        marker=dict(
            color=movies_grouped['normalized_avg_duration'],
            colorscale='Blues',
            showscale=True,
            colorbar=dict(
                title='Média de duração (min):',
                x=1.02,
                len=0.35,
                thickness=15,
                y=0.75,
                yanchor='top'
            )
        ),
        width=fixed_bar_width,
        # Combina 'avg_duration' e 'avg_release_year' em um único array para customdata.
        customdata=np.stack((movies_grouped['avg_duration'], movies_grouped['avg_release_year']), axis=-1),
        
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Total:</b> %{y}<br>' +
            '<b>Média de duração (min):</b> %{customdata[0]:.2f}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Adiciona na legenda a barra de cor referência (Tv Show) com escala (valor média em temporadas) e hover com média de release_year.
fig.add_trace(
    go.Bar(
        x=tv_shows_grouped['year_added'],
        y=tv_shows_grouped['counts'],
        name='Séries',
        marker=dict(
            color=tv_shows_grouped['normalized_avg_seasons'],
            colorscale='Reds',
            showscale=True,
            colorbar=dict(
                title='Média de temporadas:',
                x=1.02,
                len=0.35,
                thickness=15,
                y=0.4,
                yanchor='top'
            )
        ),
        width=fixed_bar_width,
        # Combina 'avg_seasons' e 'avg_release_year' em um único array para customdata
        customdata=np.stack((tv_shows_grouped['avg_seasons'], tv_shows_grouped['avg_release_year']), axis=-1),
        
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Total:</b> %{y}<br>' +
            '<b>Média de temporadas:</b> %{customdata[0]:.2f}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Calcula o valor máximo da contagem entre filmes e séries.
max_count = max(movies_grouped['counts'].max(), tv_shows_grouped['counts'].max())

# Define o limite superior do eixo Y com uma margem de 10%.
y_axis_max = max_count * 1.1

# Atualiza o layout.
fig.update_layout(
    # Propriedades de estilização do Layout.
    title='Títulos adicionados na Netflix',
    xaxis_title='Ano',
    yaxis_title='Quantidade',
    legend_title='Legenda',
    barmode='group',
    bargap=0.2,
    yaxis=dict(
        title='Quantidade',
        range=[0, y_axis_max],
        tickmode='linear',
        dtick=300
    ),
    xaxis=dict(
        title='Ano',
        tickmode='array',
        tickvals=anos
    ),
    legend=dict(
        x=1.02,
        y=1,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    ),
    margin=dict(t=80, r=200)
)

# Exibe o gráfico.
fig.show()

## Visualization model #2

In [11]:
# Define a lista de anos para o eixo X, garantindo que esteja ordenada.
anos = sorted(df['year_added'].unique())

# Criação de variáveis que recebem os campos do atributo type.
movies_df = df[df['type'] == 'Movie']
tv_shows_df = df[df['type'] == 'TV Show']

# Realiza a contagem da quantidade de filmes e a média de duração e release_year de acordo com o ano.
movies_grouped = movies_df.groupby('year_added').agg(
    counts=('title', 'count'),
    avg_duration=('duration_num', 'mean'),
    avg_release_year=('release_year', 'mean')  # Média de release_year
).reset_index()

# Realiza a contagem da quantidade de Tv Shows e a média de temporadas e release_year de acordo com o ano.
tv_shows_grouped = tv_shows_df.groupby('year_added').agg(
    counts=('title', 'count'),
    avg_seasons=('duration_num', 'mean'),
    avg_release_year=('release_year', 'mean') 
).reset_index()

# Normaliza os valores médios para o intervalo entre 0 e 1.
movies_grouped['normalized_avg_duration'] = (
    movies_grouped['avg_duration'] - movies_grouped['avg_duration'].min()
) / (movies_grouped['avg_duration'].max() - movies_grouped['avg_duration'].min())

tv_shows_grouped['normalized_avg_seasons'] = (
    tv_shows_grouped['avg_seasons'] - tv_shows_grouped['avg_seasons'].min()
) / (tv_shows_grouped['avg_seasons'].max() - tv_shows_grouped['avg_seasons'].min())

# Define uma largura fixa para as barras dos gráficos.
fixed_bar_width = 0.4

# Cria a figura (combinação do Layout e a definição dos dados).
fig = go.Figure()



# Adiciona a linha para representar quantidade de filmes.
fig.add_trace(
    go.Scatter(
        x=movies_grouped['year_added'],
        y=movies_grouped['counts'],
        mode='lines+markers',
        name='Total de filmes',
        line=dict(color='blue'),
        yaxis='y1',
        # Combina 'counts' e 'avg_release_year' em um único array para customdata.
        customdata=np.stack((movies_grouped['counts'], movies_grouped['avg_release_year']), axis=-1),
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Total de filmes:</b> %{y}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Adiciona a linha para representar a quantidade de séries.
fig.add_trace(
    go.Scatter(
        x=tv_shows_grouped['year_added'],
        y=tv_shows_grouped['counts'],
        mode='lines+markers',
        name='Total de séries',
        line=dict(color='red'),
        yaxis='y1',
        # Combina 'counts' e 'avg_release_year' em um único array para customdata.
        customdata=np.stack((tv_shows_grouped['counts'], tv_shows_grouped['avg_release_year']), axis=-1),
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Total de séries:</b> %{y}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Adiciona a linha para a duração média dos filmes.
fig.add_trace(
    go.Scatter(
        x=movies_grouped['year_added'],
        y=movies_grouped['avg_duration'],
        mode='lines+markers',
        name='Duração Média Filmes (min)',
        line=dict(color='darkblue', dash='dash'),
        yaxis='y2',
        # Combina 'avg_duration' e 'avg_release_year' em um único array para customdata.
        customdata=np.stack((movies_grouped['avg_duration'], movies_grouped['avg_release_year']), axis=-1),
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Duração Média Filmes (min):</b> %{y:.2f}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Adiciona a linha para média de temporadas das séries.
fig.add_trace(
    go.Scatter(
        x=tv_shows_grouped['year_added'],
        y=tv_shows_grouped['avg_seasons'],
        mode='lines+markers',
        name='Média de Temporadas Séries',
        line=dict(color='darkred', dash='dash'),
        yaxis='y2',
        # Combina 'avg_seasons' e 'avg_release_year' em um único array para customdata.
        customdata=np.stack((tv_shows_grouped['avg_seasons'], tv_shows_grouped['avg_release_year']), axis=-1),
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Média de Temporadas Séries:</b> %{y:.2f}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Atualiza o layout.
fig.update_layout(
    title='Títulos adicionados na Netflix',
    xaxis_title='Ano',
    xaxis=dict(
        title='Ano',
        tickmode='array',
        tickvals=anos
    ),
    yaxis=dict(
        title='Quantidade',
        range=[0, y_axis_max],
        titlefont=dict(color='black'),
        tickfont=dict(color='black')
    ),
    yaxis2=dict(
        title='Médias',
        overlaying='y',
        side='right',
        titlefont=dict(color='black'),
        tickfont=dict(color='black')
    ),
    legend_title='Legenda',
    margin=dict(t=80, r=150) 
)
# Exibe o gráfico.
fig.show()

## Visualization model #3

In [12]:
# Agrupa por 'year_added' e calcula a quantidade, a média de temporadas e a média de release_year para séries.
tv_shows_grouped = tv_shows_df.groupby('year_added').agg(
    counts=('title', 'count'),
    avg_seasons=('duration_num', 'mean'),
    avg_release_year=('release_year', 'mean')  # Média de release_year
).reset_index()

# Normaliza os valores médios para o tamanho das bolhas.
movies_grouped['marker_size'] = (
    (movies_grouped['avg_duration'] - movies_grouped['avg_duration'].min()) / 
    (movies_grouped['avg_duration'].max() - movies_grouped['avg_duration'].min())
) * 100 + 20

tv_shows_grouped['marker_size'] = (
    (tv_shows_grouped['avg_seasons'] - tv_shows_grouped['avg_seasons'].min()) / 
    (tv_shows_grouped['avg_seasons'].max() - tv_shows_grouped['avg_seasons'].min())
) * 100 + 20

# Calcula o valor máximo de 'counts' entre filmes e séries.
max_count = max(movies_grouped['counts'].max(), tv_shows_grouped['counts'].max())

# Define o limite superior do eixo Y com uma margem de 10%.
y_axis_max = max_count * 1.1

# Cria a figura.
fig = go.Figure()

# Adiciona as bolhas para filmes.
fig.add_trace(
    go.Scatter(
        x=movies_grouped['year_added'],
        y=movies_grouped['counts'],
        mode='markers',
        name='Filmes',
        marker=dict(
            size=movies_grouped['marker_size'],
            color='blue',
            sizemode='diameter',
            opacity=0.7,
            line=dict(width=2, color='blue')
        ),
        # Combina 'avg_duration' e 'avg_release_year' em um único array para customdata
        customdata=np.stack((movies_grouped['avg_duration'], movies_grouped['avg_release_year']), axis=-1),
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Total de filmes:</b> %{y}<br>' +
            '<b>Duração Média (min):</b> %{customdata[0]:.2f}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Adiciona as bolhas para séries.
fig.add_trace(
    go.Scatter(
        x=tv_shows_grouped['year_added'],
        y=tv_shows_grouped['counts'],
        mode='markers',
        name='Séries',
        marker=dict(
            size=tv_shows_grouped['marker_size'],
            color='red',
            sizemode='diameter',
            opacity=0.7,
            line=dict(width=2, color='red')
        ),
        # Combina 'avg_seasons' e 'avg_release_year' em um único array para customdata.
        customdata=np.stack((tv_shows_grouped['avg_seasons'], tv_shows_grouped['avg_release_year']), axis=-1),
        hovertemplate=(
            '<b>Ano:</b> %{x}<br>' +
            '<b>Total de séries:</b> %{y}<br>' +
            '<b>Média de Temporadas:</b> %{customdata[0]:.2f}<br>' +
            '<b>Média de ano de lançamento:</b> %{customdata[1]:.0f}<br>' +
            '<extra></extra>'
        )
    )
)
# Ajusta o layout do gráfico.
fig.update_layout(
    title='Títulos adicionados na Netflix',
    xaxis_title='Ano',
    yaxis_title='Quantidade',
    legend_title='Tipo',
    yaxis=dict(
        range=[0, y_axis_max],
        titlefont=dict(color='black'),
        tickfont=dict(color='black')
    ),
    xaxis=dict(
        title='Ano',
        tickmode='array',
        tickvals=anos
    ),
    legend=dict(
        title='Tipo',
        x=1.02,
        y=1,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    ),
    margin=dict(t=80, r=200),
    hovermode='closest'  
)
# Exibe o gráfico.
fig.show()

## Visualization model #4

In [14]:
from plotly.subplots import make_subplots
import calendar

# Cria as colunas year_added e month_added.
df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y', errors='coerce')
df = df.dropna(subset=['date_added'])  # Remover linhas com datas inválidas

df['year_added'] = df['date_added'].dt.year.astype(int)
df['month_added'] = df['date_added'].dt.month.astype(int)

# Extrai apenas os números da coluna duration.
df['duration_num'] = df['duration'].str.extract(r'(\d+)').astype(float)

# Agrupa os dados para filmes: contagem.
movies_heatmap_data = df[df['type'] == 'Movie'].groupby(['year_added', 'month_added']).size().unstack(fill_value=0)

# Garante que todos os meses de 1 a 12 estejam presentes.
movies_heatmap_data = movies_heatmap_data.reindex(columns=range(1, 13), fill_value=0)
movies_heatmap_data = movies_heatmap_data.sort_index()

# Agrupa os dados para séries: contagem.
tv_shows_heatmap_data = df[df['type'] == 'TV Show'].groupby(['year_added', 'month_added']).size().unstack(fill_value=0)

# Garante que todos os meses estejam presentes.
tv_shows_heatmap_data = tv_shows_heatmap_data.reindex(columns=range(1, 13), fill_value=0)
tv_shows_heatmap_data = tv_shows_heatmap_data.sort_index()

# Agrupa os dados para filmes: média de duração.
movies_avg_duration = df[df['type'] == 'Movie'].groupby(['year_added', 'month_added'])['duration_num'].mean().unstack(fill_value=0)

# Garante que todos os meses estejam presentes.
movies_avg_duration = movies_avg_duration.reindex(columns=range(1, 13), fill_value=0)
movies_avg_duration = movies_avg_duration.sort_index()

# Agrupa os dados para séries: média de temporadas.
tv_shows_avg_seasons = df[df['type'] == 'TV Show'].groupby(['year_added', 'month_added'])['duration_num'].mean().unstack(fill_value=0)

# Garante que todos os meses estejam presentes.
tv_shows_avg_seasons = tv_shows_avg_seasons.reindex(columns=range(1, 13), fill_value=0)
tv_shows_avg_seasons = tv_shows_avg_seasons.sort_index()

# Cria subplots 2 linhas x 2 colunas.
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Contagem de Filmes",
        "Contagem de Séries",
        "Tempo Médio dos Filmes (min)",
        "Média de Temporadas das Séries"
    ),
    shared_yaxes=True,
    vertical_spacing=0.1
)
# Heatmap para contagem de filmes.
fig.add_trace(
    go.Heatmap(
        z=movies_heatmap_data.values,
        x=[calendar.month_name[i] for i in movies_heatmap_data.columns],
        y=movies_heatmap_data.index,
        colorscale='Blues',
        showscale=False,
        hovertemplate='Ano: %{y}<br>Mês: %{x}<br>Filmes: %{z}<extra></extra>'
    ),
    row=1,
    col=1
)
# Heatmap para contagem de séries.
fig.add_trace(
    go.Heatmap(
        z=tv_shows_heatmap_data.values,
        x=[calendar.month_name[i] for i in tv_shows_heatmap_data.columns],
        y=tv_shows_heatmap_data.index,
        colorscale='Reds',
        showscale=False,
        hovertemplate='Ano: %{y}<br>Mês: %{x}<br>Séries: %{z}<extra></extra>'
    ),
    row=1,
    col=2
)
# Heatmap para tempo médio dos filmes.
fig.add_trace(
    go.Heatmap(
        z=movies_avg_duration.values,
        x=[calendar.month_name[i] for i in movies_avg_duration.columns],
        y=movies_avg_duration.index,
        colorscale='Greens',
        showscale=False,
        hovertemplate='Ano: %{y}<br>Mês: %{x}<br>Tempo Médio (min): %{z:.2f}<extra></extra>'
    ),
    row=2,
    col=1
)
# Heatmap para média de temporadas das séries.
fig.add_trace(
    go.Heatmap(
        z=tv_shows_avg_seasons.values,
        x=[calendar.month_name[i] for i in tv_shows_avg_seasons.columns],
        y=tv_shows_avg_seasons.index,
        colorscale='Purples',
        showscale=False,
        hovertemplate='Ano: %{y}<br>Mês: %{x}<br>Média de Temporadas: %{z:.2f}<extra></extra>'
    ),
    row=2,
    col=2
)
# Atualiza o layout dos gráficos.
fig.update_layout(
    title='Distribuição Mensal de Filmes e Séries Adicionados por Ano',
    xaxis_title='Mês',
    yaxis_title='Ano',
    xaxis2_title='Mês',
    xaxis3_title='Mês',
    xaxis4_title='Mês',
    yaxis3_title='Ano',
    yaxis=dict(autorange='reversed'),  
    yaxis2=dict(autorange='reversed'),
    yaxis3=dict(autorange='reversed'),
    yaxis4=dict(autorange='reversed'),
    margin=dict(t=100),
    height=800 
)
# Exibe o gráfico.
fig.show()

## Critical reflection
#### Write down 2-3 paragraphs (190-240 words) with your reflection about the obtained visualizations.

Os gráficos escolhidos para representar um conjunto de dados específicos do dataset netflix_titles.csv seguiram alguns critérios para que as visualizações pudessem mostrar de forma mais clara as tarefas relacionadas ao número de produções audivisuais adicionadas, ao número médio de reprodução e temporadas, e a idade média das mídias produzidas e adicionados a cada ano. O gráfico de barras sugere uma visualizaçao onde se compara valores categóricos (filmes x TvShows). Foi comparado o ano com a quantidade de mídias adicionados. O terceiro atributo que representa os valores da coluna 'duration' foi inserido no gráfico na forma de tonalidades das cores. Gráficos com cores mais intensos significaria tempo de reprodução (minutos) ou quantidade de temporadas maiores, e por sua vez cores menos intensas representariam o oposto (menores). Um quarto atributo foi inserido em cada célula do gráfico para mostrar a idade média dos filmes produzidos num determinado ano. O gráfico de linhas sugere uma visualização de tendência dos dados ao longo do tempo. Foi inserido quatro linhas para fazer a representação da quantidade de mídias adicionadas e o tempo médio em minutos para filmes e média de temporadas para TVShows. O gráfico de bolhas sugere uma visualização tridimensional que facilita a identificação de padrões baseados no tamanho e na cor das bolhas. O gráfico de mapa de calor sugere uma visualização de dados baseada na tonalidade de cor de cada célula. É ideal para identificar padrões de distribuição ao longo de dois eixos (ano e mês).

## References
#### All the sources you have used in this project should cited and credited in this section.

[https://datos.gob.es/sites/default/files/doc/file/data_visualization_tool_report.pdf]
[https://www.di.fc.ul.pt/~tc/papers/2013_EPCG_VisTempVideo.pdf]
[https://www.alea.pt/images/dossies_pdf/dossier9.pdf]
[https://infodesign.org.br/public/journals/1/V7_n2_2010/ID_v7_n2_2010_01_06_Portugal.pdf?download=1&phpMyAdmin=H8DwcFLEmv4B1mx8YJNY1MFYs4e]
